In [48]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload
import time
from skfuzzy import cmeans
import artificial_bee_colony.colonies as cols
import artificial_bee_colony.goals as goals
import abc_plusplus
import abc_plusplus_256dim
import abc_plusplus_128dim
import abc_plusplus_exp128
from skopt import BayesSearchCV
import sklearn
from skopt.space import Real, Categorical, Integer
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import matplotlib.pyplot as plt
from sklearn import metrics
import scipy
import itertools
import random
from scipy.spatial import distance

def load_pa(filename):
    result = []
    with open(filename) as f:
        for line in f:
            if line[:5] == '-----':
                break
        for line in f:
            result.append(int(line))
    return result

In [145]:
def map_labels(original, generated):
    org_values = list(set(original))
    org_i_mapping = {org_values[i]:i for i in range(len(org_values))}

    gen_values = list(set(generated))
    gen_i_mapping = {gen_values[i]:i for i in range(len(gen_values))}

    cost_matrix = np.zeros([len(org_values), len(gen_values)])
    for i in range(len(org_values)):
        indices = np.where(original == org_values[i])[0]
        counts = Counter(generated[indices])
        for k in counts:
            cost_matrix[i][gen_i_mapping[k]] = -counts[k]
    
    org_i, gen_i = scipy.optimize.linear_sum_assignment(cost_matrix)
    gen_i = gen_i[org_i.argsort()]
    org_i = np.sort(org_i)
    
    r = []
    for g in generated:
        for i, j in zip(gen_i, org_i):
            if g == gen_values[i]:
                r.append(org_values[j])
                
    return r

def map_fuzzy_labels(original_f, generated_f, padding=True):
    original = original_f.argmax(axis=0)
    generated = generated_f.argmax(axis=0)
    
    org_values = list(set(original))
    org_i_mapping = {org_values[i]:i for i in range(len(org_values))}

    gen_values = list(set(generated))
    gen_i_mapping = {gen_values[i]:i for i in range(len(gen_values))}

    cost_matrix = np.zeros([len(org_values), len(gen_values)])
    for i in range(len(org_values)):
        indices = np.where(original == org_values[i])[0]
        counts = Counter(generated[indices])
        for k in counts:
            cost_matrix[i][gen_i_mapping[k]] = -counts[k]
    
    org_i, gen_i = scipy.optimize.linear_sum_assignment(cost_matrix)
    gen_i = gen_i[org_i.argsort()]
    org_i = np.sort(org_i)
    
    r = []
    for i in gen_i:
        r.append(generated_f[org_i[i]])
        
    if len(org_values) > len(gen_values):
        for i in range(len(org_values) - len(gen_values)):
            r.append(np.zeros(generated_f.shape[1]))
                
    return np.array(r)

def compute_cost(weights, data):
    result = 0
    for cluster in weights:
        center = sum((data.T * cluster).T) / sum(cluster)
        for vector, weight in zip(data, cluster):
            result += weight * distance.euclidean(vector, center)
    return result

In [3]:
dimset_256 = np.loadtxt('data/dim_set/dim256.txt')
dimset_256_partitions = load_pa('data/dim_set/dim256.pa.txt')

In [108]:
start = time.process_time()
cntr, u, _, _, _, _, _ = cmeans(dimset_256.T, 16, 2, 0.005, 20_000)
print(time.process_time() - start)

0.03125


In [139]:
u

array([[0.06250211, 0.06250212, 0.06250213, ..., 0.06249244, 0.06249245,
        0.06249243],
       [0.06250796, 0.06250798, 0.06250798, ..., 0.06250777, 0.06250777,
        0.06250781],
       [0.06251134, 0.06251137, 0.06251137, ..., 0.06253415, 0.06253415,
        0.06253428],
       ...,
       [0.06251303, 0.06251303, 0.06251305, ..., 0.062505  , 0.062505  ,
        0.06250498],
       [0.06250835, 0.06250837, 0.06250833, ..., 0.06251198, 0.06251199,
        0.06251206],
       [0.06249112, 0.06249111, 0.06249109, ..., 0.06246837, 0.06246837,
        0.0624683 ]])

In [118]:
metrics.accuracy_score(dimset_256_partitions, u.argmax(axis=0))

0.0625

In [47]:
colony = abc_plusplus_256dim.ArtificialBeeColony(dimset_256, 16, 20, 200)
start = time.process_time()
cpp_result = np.array(colony.optimize(3000))
print(time.process_time() - start)

2761.515625


In [104]:
cpp_result = np.array(colony.optimize(3000))

In [105]:
metrics.homogeneity_score(dimset_256_partitions, cpp_result.argmax(axis=0))

0.15060910069892866

In [106]:
metrics.completeness_score(dimset_256_partitions, cpp_result.argmax(axis=0))

0.15105038696481127

In [107]:
metrics.v_measure_score(dimset_256_partitions, cpp_result.argmax(axis=0))

0.15082942106136807

In [108]:
metrics.accuracy_score(dimset_256_partitions, cpp_result.argmax(axis=0))

0.05078125

In [103]:
cpp_result

array([[0.14427302, 0.07975287, 0.06959584, ..., 0.13732012, 0.02405726,
        0.04995593],
       [0.03975074, 0.        , 0.05739762, ..., 0.06667717, 0.14101816,
        0.02703793],
       [0.09437256, 0.15799522, 0.08657185, ..., 0.06545022, 0.01349141,
        0.04662401],
       ...,
       [0.03580701, 0.09186799, 0.1093345 , ..., 0.01202827, 0.10953663,
        0.01497368],
       [0.04164424, 0.00872534, 0.06486476, ..., 0.04870205, 0.09694321,
        0.        ],
       [0.08422256, 0.        , 0.0018656 , ..., 0.03954207, 0.10830947,
        0.02633559]])

In [129]:
start = time.process_time()
b_cntr, b_u, _, _, _, _, _ = cmeans(dimset_256.T, 16, 2, 0.005, 1000, init=cpp_result)
print(time.process_time() - start)

0.03125


In [130]:
b_u

array([[0.06251265, 0.06251267, 0.0625127 , ..., 0.06249429, 0.06249429,
        0.06249428],
       [0.06247038, 0.06247034, 0.06247031, ..., 0.06251972, 0.06251971,
        0.06251986],
       [0.0624954 , 0.0624954 , 0.06249539, ..., 0.06248523, 0.06248525,
        0.06248519],
       ...,
       [0.06249095, 0.06249096, 0.06249092, ..., 0.06248621, 0.06248623,
        0.06248621],
       [0.06249486, 0.06249484, 0.06249483, ..., 0.06249458, 0.06249457,
        0.06249454],
       [0.0624937 , 0.0624937 , 0.06249369, ..., 0.06249402, 0.062494  ,
        0.06249401]])

In [131]:
metrics.accuracy_score(dimset_256_partitions, b_u.argmax(axis=0))

0.0625

In [51]:
dimset_128 = np.loadtxt('data/dim_set/dim128.txt')
dimset_128_partitions = load_pa('data/dim_set/dim128.pa.txt')
dimset_128_fuzzy_gt = np.load('data/dim_set/dim128-fuzzy_gt.npy')

In [109]:
start = time.process_time()
cntr, u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 20_000)
print(time.process_time() - start)

51.15625


In [110]:
Counter(u.argmax(axis=0))

Counter({0: 1024})

In [111]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), u.argmax(axis=0)))

0.0625

In [165]:
u

array([[0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       ...,
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625],
       [0.0625, 0.0625, 0.0625, ..., 0.0625, 0.0625, 0.0625]])

In [163]:
#np.save('cmeans_experiment1.npy', u)

In [146]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, u))

0.062255859374999986

In [164]:
compute_cost(u, dimset_128)

601614.9660454898

In [149]:
start = time.process_time()
cntr, u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 100_000)
print(time.process_time() - start)

256.609375


In [150]:
#np.save('cmeans_experiment2.npy', u)

In [151]:
Counter(u.argmax(axis=0))

Counter({0: 1024})

In [152]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), u.argmax(axis=0)))

0.0625

In [153]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, u))

0.062255859374999986

In [161]:
compute_cost(u, dimset_128)

601614.9660454898

In [39]:
colony = abc_plusplus_exp128.ArtificialBeeColony(dimset_128, 16, 30, 317)
start = time.process_time()
cpp_result = np.array(colony.optimize(20_000))
print(time.process_time() - start)

14202.171875


In [40]:
Counter(cpp_result.argmax(axis=0))

Counter({14: 65,
         0: 62,
         2: 63,
         5: 51,
         10: 69,
         11: 72,
         7: 90,
         3: 65,
         1: 63,
         13: 55,
         15: 57,
         12: 94,
         8: 58,
         9: 64,
         6: 46,
         4: 50})

In [41]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), cpp_result.argmax(axis=0)))

0.5302734375

In [45]:
#np.save('ABC_experiment1.npy', cpp_result)

In [101]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, cpp_result))

0.04194288250371845

In [160]:
compute_cost(cpp_result, dimset_128)

543142.3306296628

In [166]:
start = time.process_time()
b_cntr, b_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.0, 1000, init=cpp_result)
print(time.process_time() - start)

2.5625


In [167]:
Counter(b_u.argmax(axis=0))

Counter({2: 64,
         7: 64,
         0: 64,
         11: 64,
         3: 64,
         4: 64,
         10: 64,
         8: 64,
         12: 64,
         6: 64,
         9: 64,
         1: 64,
         15: 64,
         13: 64,
         14: 64,
         5: 64})

In [168]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), b_u.argmax(axis=0)))

1.0

In [169]:
metrics.mean_squared_error(dimset_128_fuzzy_gt.T, map_fuzzy_labels(dimset_128_fuzzy_gt.T, b_u))

7.875039027644074e-06

In [170]:
compute_cost(b_u, dimset_128)

18465.50909884966

In [15]:
mod_colony = abc_plusplus_128dim.ModArtificialBeeColony(dimset_128, 16, 23, 95, 0.4891644921988578, 0.11924786590959738)
start = time.process_time()
mod_cpp_result = np.array(mod_colony.optimize(6000))
print(time.process_time() - start)

3379.421875


In [16]:
Counter(mod_cpp_result.argmax(axis=0))

Counter({15: 92,
         4: 101,
         14: 82,
         2: 67,
         12: 85,
         3: 68,
         10: 79,
         7: 73,
         6: 52,
         8: 50,
         1: 48,
         13: 45,
         11: 49,
         9: 43,
         0: 43,
         5: 47})

In [17]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), mod_cpp_result.argmax(axis=0)))

0.5380859375

In [18]:
start = time.process_time()
mb_cntr, mb_u, _, _, _, _, _ = cmeans(dimset_128.T, 16, 2, 0.005, 1000, init=mod_cpp_result)
print(time.process_time() - start)

0.0625


In [19]:
metrics.accuracy_score(dimset_128_partitions, map_labels(np.array(dimset_128_partitions), mb_u.argmax(axis=0)))

1.0

In [22]:
def create_estimator(data):
    class ABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.ArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return ABCEstimator

def create_tournament_estimator(data):
    class TournamentABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_exp128.TournamentArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return TournamentABCEstimator

In [4]:
sample = np.array(random.sample(range(len(dimset_128)), k=258))

In [12]:
sample

array([ 326,   58,   10,  210,  517,  264, 1001,  204,  332,  894,  616,
        526,  758,  548,   97,  440,  698,  438,  924,  721,  716,  684,
        159,  584,  644,  661,  469,  265,  511,  572,  311,  792,  173,
        656,  606,  415,  729,   21,   85,  532,  573,  845,  460,  983,
         51,   56,  920,  269,  103,  467, 1004,  903,  599,  663,  263,
        519,  586,  211,  447,  622,  551,  915,  744,  867, 1002,  872,
       1018,  431,  892,  107,   49,   31,  221,  933,  723,  244,  725,
        396,   81,  328,  143,  897,  726,  574,  890,  377,  385,  675,
        418,  563,  695,  222,  386,  596,  361,  627,  544,  368,  543,
        687,  623,  938,  921,  557,  546,  769,  250,  806,  805,  270,
        885,  889,  961,  126,  378,  705,  445,  240,  884,  112,  251,
         65,  142, 1013,  626,  967,  505,  437,  243,   43,  570, 1011,
        771,  303,  746,  646,  972,  369,  620,  395,  503,  156,  100,
        459,  151,  597,  174,   52,  278,  207, 10

In [14]:
sample_data = dimset_128[sample]
sample_labels = np.array(dimset_128_partitions)[sample]

In [6]:
Counter(sample_labels).most_common()

[(5, 20),
 (10, 19),
 (7, 19),
 (14, 18),
 (15, 18),
 (6, 17),
 (4, 17),
 (9, 17),
 (13, 16),
 (12, 15),
 (2, 15),
 (1, 14),
 (16, 14),
 (11, 13),
 (3, 13),
 (8, 13)]

In [31]:
estimator = create_estimator(sample_data)(20, 1000, 200)
bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=10, verbose=1)

In [32]:
bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.3min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.6min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=10, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [33]:
bayes.best_params_

{'limit': 309, 'pop': 50}

In [18]:
estimator = create_estimator(sample_data)(20, 1000, 200)
bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=30, verbose=1)

bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.8min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.7min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.1s finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [20]:
bayes.best_params_

{'limit': 317, 'pop': 30}

In [23]:
estimator = create_tournament_estimator(sample_data)(20, 1000, 200)
t_bayes = BayesSearchCV(estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400)
}, n_iter=30, verbose=1)

t_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.6min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.3min finished


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   56.1s finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\skopt\optimizer\optimizer.py:399: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   55.9s finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=TournamentABCEstimator(cycles=1000, limit=200, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400)},
       verbose=1)

In [24]:
t_bayes.best_params_

{'limit': 25, 'pop': 50}

In [26]:
def create_mod_estimator(data):
    class ModABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, f=0.8, mr=0.1, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.f = f
            self.mr = mr
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_128dim.ModArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit), float(self.f), float(self.mr))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return ModABCEstimator

def create_tournament_mod_estimator(data):
    class TournamentModABCEstimator(sklearn.base.BaseEstimator):
        def __init__(self, pop=20, cycles=1000, limit=200, f=0.8, mr=0.1, n_clusters=16):
            self.pop = pop
            self.cycles = cycles
            self.limit = limit
            self.f = f
            self.mr = mr
            self.n_clusters = n_clusters

        def fit(self, data, y=None):
            self._colony = abc_plusplus_exp128.TournamentModArtificialBeeColony(data, self.n_clusters, int(self.pop), int(self.limit), float(self.f), float(self.mr))
            self._colony.fit(int(self.cycles))
            return self

        def score(self, data):
            return self._colony.score()
    return TournamentModABCEstimator

In [9]:
mod_estimator = create_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=15, verbose=1)

In [10]:
mod_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.3min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.7min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=15, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [11]:
mod_bayes.best_params_

{'f': 0.4891644921988578, 'limit': 95, 'mr': 0.11924786590959738, 'pop': 23}

In [15]:
mod_estimator = create_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=30, verbose=1)

mod_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.2min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  4.5min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.2min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=ModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7, n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [16]:
mod_bayes.best_params_

{'f': 0.41230542901888145, 'limit': 187, 'mr': 0.18862883068061148, 'pop': 41}

In [27]:
mod_estimator = create_tournament_mod_estimator(sample_data)(20, 1000, 200, 0.1, 0.7, 16)
mod_t_bayes = BayesSearchCV(mod_estimator, {
    'pop': Integer(10, 50),
    'limit': Integer(20, 400),
    'f': Real(0, 1),
    'mr': Real(0, 1),
}, n_iter=30, verbose=1)

mod_t_bayes.fit(sample_data)

c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  2.7min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.5min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  5.6min finished
c:\users\jakub\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.1min finished


BayesSearchCV(cv=None, error_score='raise',
       estimator=TournamentModABCEstimator(cycles=1000, f=0.1, limit=200, mr=0.7,
             n_clusters=16, pop=20),
       fit_params=None, iid=True, n_iter=30, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False, scoring=None,
       search_spaces={'pop': Integer(low=10, high=50), 'limit': Integer(low=20, high=400), 'f': Real(low=0, high=1, prior='uniform', transform='identity'), 'mr': Real(low=0, high=1, prior='uniform', transform='identity')},
       verbose=1)

In [28]:
mod_t_bayes.best_params_

{'f': 0.9995027160733371, 'limit': 393, 'mr': 0.00852256385047623, 'pop': 49}

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#